<a href="https://colab.research.google.com/github/srirammk/DLTest/blob/main/ImageConverter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#lightweight model to convert 2d image to 3D image
import torch
import cv2
import numpy as np
from torchvision.transforms import Compose, ToTensor, Normalize


In [5]:
class Circuit2DTo3D:
    def __init__(self):
        self.model_type = "midas_v21_small"
        self.midas = torch.hub.load("intel-isl/Midas", self.model_type)
        self.device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
        self.midas.to(self.device)
        self.midas.eval()

        self.transform = Compose([
            ToTensor(),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def predict_depth(self, img):
        img_input = self.transform(img)
        img_input = img_input.unsqueeze(0).to(self.device)
        prediction = self.midas(img_input)
        return torch.nn.functional.interpolate(
            prediction.unsqueeze(1),
            size=img.shape[:2],
            mode="bicubic",
            align_corners=False
        ).squeeze().cpu().numpy()

    def visualize_depth(self, depth_map):
        x_ = np.nan_to_num(depth_map) # Change nan to 0
        mi = np.min(x_) # Get minimum depth
        ma = np.max(x_) # Get maximum depth
        x_ = (x_-mi)/(ma-mi+1e-8) # Normalize to 0-1
        x_ = (255*x_).astype(np.uint8)
        return cv2.applyColorMap(x_, cv2.COLORMAP_JET)